In [1]:
import pandas as pd 
import ast
import json
from graph_tool.all import *
import numpy as np
import geopy.distance as gp



/home/ikenna/anaconda3/lib/python3.7/site-packages/graph_tool/draw/cairo_draw.py:1494: RuntimeWarning: Error importing Gtk module: No module named 'gi'; GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)


## In this program, we convert data of  nodes and actions into a graph. The graph will be used to build the routing algorithm

In [ ]:
# read in data
with open("state_action.json","r+") as content:
    ways = json.load(content)
    ways = {ast.literal_eval(k):v for k,v in ways.items() }

In [ ]:
df = pd.read_csv("safety_data.csv", names=["ratings","scores","voice_info","lon","lat"])

# get ratings of coordinates
def get_rating(point,data=df[["lon","lat"]].to_numpy(),rating=df.ratings):
    diff = np.hypot(*(data - point).T)
    index =  np.argmin(diff)
    return rating[index]

# distance between 2 coordinates
def get_distance(x,y):
    p1 = (x[1],x[0])
    p2 = (y[1],y[0])
    return gp.distance(p1,p2).km

In [45]:
# get a set of all nodes
nodes = set()
for v in ways.values():
    v = [ tuple(el) for el in v]
    nodes.update(v)
nodes.update(ways.keys())

In [46]:
# construct graph and graph properties
g = Graph()
coord = g.new_vertex_property("vector<double>")
weight = g.new_edge_property("float")

In [47]:
# keep track of nodes we have encountered before
# counter was much faster than always searching for node within graph
counter = {k:0 for k in nodes}

vert_dict=dict() # dictionary of vertex id for each point
point_list = list(ways.keys())

for i in range(len(point_list)):
    #for each nodes get the reachable nodes, add them to graph, and draw an edge between them
    point1 = point_list[i]
    for point2 in ways[point1]:
        point2 = tuple(point2)
        
        # we assign the weight to edges by looking at the distance and safety rating
        # we want least safe roads to have harshest penalties
        distance = get_distance(point1,point2)
        rating = get_rating(point1)
        # we want to avoid the highest risks ig possible
        if rating>2:
            r = rating**2
            w = distance*r
            # for medium risks we have a much lighter penalty
        elif rating == 2:
            r = rating**0.5
            w = distance* r
        else:
            r = rating
            w = distance*r
          
        
        # if we have note encountered node before, create a vertex, add to vertex dictionary and add node property
        if counter[point1]==0:
            u = g.add_vertex()
            vert_dict[point1] = u
            counter[point1] = 1
            coord[u] = point1
        # retrieve vertex id
        else:
            u = vert_dict[point1]
            
        # if we have note encountered node before, create a vertex, add to vertex dictionary and add node property
        if counter[point2]==0:
            v = g.add_vertex()
            vert_dict[point2] = v
            coord[v] = point2
            counter[point2] = 1
        # retrieve vertex id
        else:
            v = vert_dict[point2]      
        
        
        # add edge
        e = g.add_edge(u,v)
        # add weight to edge
        weight[e] = w  
        
# assign graph properties        
g.vertex_properties["coord"] = coord
g.edge_properties["weight"] = weight
        

In [49]:
# save output
g.save("my_graph.xml.gz")